## <span style="color:red">Metaprogramming in Python</span>

Con questo termine (<em>metaprogramming</em>) si intende la possibilità che un programma ha di "manipolare" (cioè cambiare, adattare) se stesso a tempo di esecuzione. 
Abbiamo già visto qualcosa in tal senso, ma in questo notebook vogliamo mettere meglio a fuoco il concetto

#### Un primo esempio, utilizzando solo meccanismi già presi in considerazione

In [ ]:
def a_method(self,*args,**kwargs):
    '''Una funzione, che potrebbe diventare un metodo di una qualche classe...'''
    print("A do-nothing implementation")
    return True

# La seguente porzione di codice è il più semplice esempio di possibili situazioni
# in cui le funzionalità complete (in questo caso, come vedremo) di una classe
# devono o possono solo essere decise a tempo di esecuzione, in quanto dipendenti
# da informazioni fornite dall'utente (o presenti in file di configurazione).

answer = input("The class must contain a_method? (Y/n): ")
if not answer or answer.upper()=='Y':
    include = True
else:
    include = False
    
def customize(cls):
    '''Customization function'''
    if include:
        cls.another_method = a_method
    return cls

In [ ]:
# Alla luce di quanto presentato sopra potremmo ad esempio scrivere
@customize
class a_class:
    pass

In [ ]:
a = a_class()
a.another_method()

### Verso le metaclassi

In Python3 (come più volte ricordato) è stata portata a completezza l'unificazione dei tipi definiti da utente e dei "tipi primitivi"

1. Ogni oggetto è istanza di una qualche classe
2. Ogni entità manipolabile nel linguaggio è un oggetto
3. Quindi anche le classi sono oggetti
4. Ne consegue che, per ogni oggetto x, sarà sempre vero che type(x)==x.\_\_class\_\_ (laddove la seconda scrittura si può impiegare)

In [ ]:
type(int)==int.__class__

In [ ]:
class foo:
    pass
type(foo) == foo.__class__

In [ ]:
x = foo()
type(x) == x.__class__

In [ ]:
type((3,3)) == (3,3).__class__

In [ ]:
type({'A':1, 'B':2}) == {'A':1, 'B':2}.__class__

In [ ]:
# Ma...
type(3) == 3.__class__

In [ ]:
# Se x è istanza di una class X, allora è chiaro che ...
type(x)

__Attenzione__: la "forma" del precedente output è dovuta a ipython (cioè, al fatto che ipython "intercetta" l'output dell'interprete python e, talvolta, lo elabora). Se "by-passiamo" ipython mediante stampa dell'output, vediamo il risultato che ci darebbe l'interprete

In [ ]:
print(type(x))

Dato che anche le classi sono oggetti, ci chiediamo ora quale sia il <u>tipo di una classe</u>

In [ ]:
print(type(foo))

__type__ è il tipo di ogni classe (attenzione anche qui a non confondere il concetto di tipo di una classe con quello di superclassi di una classe)

type è (naturalmente) anch'esso una classe il cui tipo è type stesso

In [ ]:
type(type)

__Una logica deduzione__. Se:

1. type è una classe
2. le istanze di type sono classi

allora la scrittura 

type() 

dovrebbe essere consentita e, come risultato, produrre una classe! 

E in effetti lo è, ma quale classe produce? Con quale nome?

Per rispondere in modo semplice, possiamo dire che type (in quanto classe) possiederà un metodo un metodo __init__; questo accetta tre parametri: nome della classe, superclassi e un dizionario che "confluirà" poi nel \_\_dict\_\_ della classe 

#### Esempio 1

In [ ]:
d = {'A':1, 'B':2}
T1 = type('X',(),d)

In [ ]:
T1.__dict__

In [ ]:
x = T1()

In [ ]:
x.A

In [ ]:
x.B

In [ ]:
type(x)

#### Esempio 2

In [ ]:
def myinit(self,a,b):
    self.a = a
    self.b = b

T2 = type('Y', (), {'__init__': myinit})

In [ ]:
y = T2('pippo', 'pluto')

In [ ]:
y.a

#### Esempio 3

In [ ]:
def mynew(cls,t,w=0):
    self = cls.__bases__[0].__new__(cls,t) #cls.__bases__[0] equivale a super()
    self.w = w
    return self

In [ ]:
T3 = type('mytuple',(tuple,),dict(__new__= mynew))

In [ ]:
z = T3((1,2),2)

In [ ]:
z

In [ ]:
z.w

In [ ]:
type(z)

## Metaclassi

Detto in parole semplici, le metaclassi sono il meccanismo mediante il quale è possibile "customizzare" l'istanziazione di una classe.

Una classe è un'istanza di type che, abbiamo visto, è istanza di se stessa. Dunque type è una __metaclasse__ e sono metaclassi tutte le classi che ereditano da type

In [ ]:
class una_metaclasse(type):
    def __init__(cls, name, bases, dictionary):
        '''Inizializzazione che prevede solo stampe, per capire
           quando viene eseguita'''
        print(cls)
        print(name)
        print(bases)
        print(dictionary)
    foo = 0 #Ci servirà dopo....

Possiamo fare in modo che il tipo di una classe sia una metaclasse diversa da type utilizzando la seguente sintassi

```python
class NOMECLASSE(ELENCO_SUPERCLASSI,metaclass=NOMEMETACLASSE):
    CORPODELLACLASSE
```

Esempio

In [ ]:
class S:
    foo = 10

class X(S,metaclass=una_metaclasse):
    pass

Che cosa è successo?

Quando viene creata una classe (in questo caso X) il cui tipo è una metaclasse diversa da type:

1. vengono chiamati i metodi \_\_new\_\_ e \_\_init\_\_ della metaclasse
2. poichè (naturalmente) valgono i meccanismi di ereditarietà e la metaclasse eredita da type, se la metaclasse non definisce uno di tali metodi allora viene chiamato il metodo corrispondente di type 

Nel nostro caso una_metaclasse definisce \_\_init\_\_ ma non \_\_new\_\_ quindi è stato prima chiamato il metodo \_\_new\_\_ di type e poi il metodo \_\_init\_\_ di una_metaclasse

Naturalmente possiamo anche scrivere il metodo \_\_new\_\_

In [ ]:
class una_metaclasse(type):
    def __new__(cls, name, bases, dictionary):
        '''In questa versione "intercettiamo" __new__ e chiamiamo esplicitamente
           la __new__ di type'''
        print(cls)
        print(name)
        print(bases)
        print(dictionary)
        return type.__new__(cls, name, bases, dictionary)
    foo = 10
        
class X(S, metaclass=una_metaclasse):
    '''X eredita da tuple ma il suo tipo è una_metaclasse'''
    foo = 5
    pass

In [ ]:
Qualche esercizio per capire se abbiamo capito...

In [ ]:
x = X()
# Quali saranno i valori stampati dai seguenti comandi?
print(x.foo)
print(type(x).foo)
print(type(X).foo)
print(type(type(x)).foo)

Continua....